In [ ]:
import os
from src.llama import llama
from src.postgres_db import VectorDB

OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://ollama:11434")
MODEL = "llama3.2:1b" # Find available models here https://ollama.com/library
EMBEDDING_MODEL = "granite-embedding:30m" # Find available models here https://ollama.com/library

POSTGRES_HOST = os.getenv("POSTGRES_HOST", "vector-postgres")

vectordb = VectorDB(host=POSTGRES_HOST)
llm = llama(OLLAMA_HOST, model=MODEL, embedding_model=EMBEDDING_MODEL)

Model 'llama3.2:1b' not found. Pulling from Ollama...
Model 'llama3.2:1b' pulled successfully.
Model 'granite-embedding:30m' is already available in Ollama.


## Generate Response

In [32]:
prompt = "How much do github actions cost?"
embedding = llm.create_embedding(f"{prompt}")
similar_docs = vectordb.search_similar(embedding, top_k=3)
context=""
for doc in similar_docs:
    context += doc[1] + "\n"

prompt_with_context = f"Context: {context}\n\nQuestion: {prompt}\n"

response = llm.generate_response(prompt_with_context)
print(response if response else "No response generated.")

The text does not explicitly state the cost of GitHub Actions. However, it provides some information about how to calculate costs and what is included in a subscription.

According to the text, GitHub calculates the cost of minutes and storage used over the amount included in an account. The cost is calculated based on hourly usage during that month.

The text also mentions that some Service features are billed based on your usage, and monthly payment for these purchases will be charged on a periodic basis in arrears, provided that for invoiced customers, paid Service features are billed in advance.

To estimate the costs, you can use the GitHub pricing calculator, which allows you to calculate the cost of minutes and storage used over the amount included in your account. Additionally, you may want to consider factors such as usage-based billing, payment terms, and subscription licenses when planning for costs.

It's also worth noting that GitHub provides some additional information on